# [실습 프로젝트] Naive RAG 구현 

- 각 단계별 지시사항에 따라 코드를 완성하세요. 
- 제시된 지시사항과 LangChain 문서를 참조하여 시스템을 구성합니다. 

In [ ]:
# 3단계: 문서 관리

# 새로운 문서 추가
new_doc = Document(
    page_content="쿠버네티스 클러스터 운영 가이드",
    metadata={"type": "tutorial", "author": "김미영"}
)
new_id = str(uuid.uuid4())
practice_db.add_documents(documents=[new_doc], ids=[new_id])
print(f"새 문서 추가 완료: {new_id}")

# 특정 문서 삭제 (첫 번째 문서 삭제)
delete_id = doc_ids[0]
practice_db.delete(ids=[delete_id])
print(f"문서 삭제 완료: {delete_id}")

`(1) 벡터 저장소 설정`
- HuggingFace에서 지원하는 BAAI/bge-m3 임베딩 모델을 사용하여 문서를 벡터화
- FAISS DB를 벡터 스토어로 사용 (IndexFlatL2 사용: 유클리드 거리)

In [ ]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings  

# Hugging Face의 임베딩 모델 생성
# 힌트: HuggingFaceEmbeddings(model_name="BAAI/bge-m3") 사용
embeddings_model = None

# 임베딩 차원 확인
embedding = embeddings_model.embed_query("test")
print(f"임베딩 차원: {len(embedding)}")

In [ ]:
# Ollama 임베딩 모델을 사용한 FAISS 벡터 저장소 생성
import faiss 
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

# FAISS 인덱스 초기화 (유클리드 거리 사용)
dim = 1024  # 임베딩 차원
faiss_index = faiss.IndexFlatL2(dim)  

# FAISS 벡터 저장소 생성
faiss_db = None

# 저장된 문서의 갯수 확인
print(faiss_db.index.ntotal)

In [ ]:
import uuid

# 문서 id 생성
doc_ids = [str(uuid.uuid4()) for _ in range(len(chunks))]

# 문서를 벡터 저장소에 저장
# 힌트: faiss_db.add_documents(chunks, ids=doc_ids) 사용
added_doc_ids = None

# 벡터 저장소에 저장된 문서를 확인
print(f"{len(added_doc_ids)}개의 문서가 성공적으로 벡터 저장소에 추가되었습니다.")
print(added_doc_ids)

`(2) 검색기 정의`
- mmr 검색으로 상위 3개 문서 검색하는 Retriever 사용
- 다양성을 높이는 설정을 사용 

In [ ]:
# mmr 검색기 생성
# 힌트: faiss_db.as_retriever(search_type='mmr', search_kwargs={'k': 3, 'fetch_k': 10, 'lambda_mult': 0.3})
# lambda_mult를 낮게 설정하여 다양성을 높임
faiss_mmr_retriever = None

In [ ]:
# 검색 테스트 
query = "대표적인 시퀀스 모델은 어떤 것들이 있나요?"
# 힌트: faiss_mmr_retriever.invoke(query) 사용
retrieved_docs = None

print(f"쿼리: {query}")
print("검색 결과:")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"-{i}-\n{doc.page_content[:100]}...{doc.page_content[-100:]}")
    print("-" * 100)

`(3) RAG 프롬프트 구성`

- 작성 기준: 
    - LangChain의 ChatPromptTemplate 클래스 사용
    - 변수 처리는 {context}, {question} 형식 사용
    - 답변은 한글로 출력되도록 프롬프트 작성
    
- 아래 템플릿 코드를 기반으로 다음 내용을 참고하여 작성합니다. 

    1. 프롬프트 구성요소:
        - 작업 지침
        - 컨텍스트 영역
        - 질문 영역
        - 답변 형식 가이드

    2. 작업 지침:
        - 컨텍스트 기반 답변 원칙
        - 외부 지식 사용 제한
        - 불확실성 처리 방법
        - 답변 불가능한 경우의 처리 방법

    3. 답변 형식:
        - 핵심 답변 섹션
        - 근거 제시 섹션
        - 추가 설명 섹션 (필요시)

    4. 제약사항 반영:
        - 답변은 사실에 기반해야 함
        - 추측이나 가정을 최소화해야 함
        - 명확한 근거 제시가 필요함
        - 구조화된 형태로 작성되어야 함

In [ ]:
# Prompt 템플릿 (예시)
from langchain_core.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context.

[Context]
{context}

[Question] 
{question}

[Answer]
"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
# Prompt 템플릿 (여기에 작성하세요)
from langchain_core.prompts import ChatPromptTemplate

template = None

prompt = ChatPromptTemplate.from_template(template)

# 템플릿 출력
prompt.pretty_print()

`(4) RAG 체인 구성`
- LangChain의 LCEL 문법을 사용
- 검색 결과를 프롬프트의 'context'로 전달하고,
- 사용자가 입력한 질문을 그래도 프롬프트의 'question'에 전달
- LLM 설정:
    - ChatOpenAI 사용 ('gpt-4o-mini' 모델)
    - temperature: 답변의 일관성을 가져가는 설정값을 사용 
    - 기타 필요한 설정 
- 출력 파서: 문자열 부분만 출력되도록 구성

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# LLM 설정
# 힌트: ChatOpenAI(model='gpt-4o-mini', temperature=0) 사용
llm = None

# 문서 포맷팅
def format_docs(docs):
    return "\n\n".join([f"{doc.page_content}" for doc in docs])

# RAG 체인 생성
# 힌트: {'context': faiss_mmr_retriever | format_docs, 'question': RunnablePassthrough()} | prompt | llm | StrOutputParser()
rag_chain = None

# 체인 실행
query = "대표적인 시퀀스 모델은 어떤 것들이 있나요?"
output = None

print(f"쿼리: {query}")
print("답변:")
print(output)

`(5) Gradio 스트리밍 구현`
- ChatInterface 사용
- `chain.stream()`으로 응답을 청크 단위로 스트리밍

In [ ]:
import gradio as gr
from typing import Iterator

# 스트리밍 응답 생성 함수
def get_streaming_response(message: str, history) -> Iterator[str]:
    
    # RAG Chain 실행 및 스트리밍 응답 생성
    response = ""
    for chunk in rag_chain.stream(message):
        if isinstance(chunk, str):
            response += chunk
            yield response

# Gradio 인터페이스 설정
# 힌트: gr.ChatInterface(fn=get_streaming_response, title="RAG 기반 질의응답 시스템", description="...", examples=[...])
demo = None

# 실행
demo.launch()

In [ ]:
# demo 실행 종료
demo.close()